In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./make.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.moogparts.com/driv/partfinder/api.catalog.models?brand=fllj&locale=en_US&country_code=US&year_id=' + input_.loc[a, 'Year_Id'] + '&make_id=' + input_.loc[a, 'Make_Id'] + '&vehicle_group_ids=2',
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Year_Id': int(input_.loc[a, 'Year_Id']),
                                        'Year': input_.loc[a, 'Year'],
                                        'Make_Id': int(input_.loc[a, 'Make_Id']),
                                        'Make': input_.loc[a, 'Make'],
                                        'Model_Id': [model['id'] for model in json_data['models']],
                                        'Model': [model['value'] for model in json_data['models']]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Year_Id': int(input_.loc[a, 'Year_Id']),
                                     'Year': input_.loc[a, 'Year'],
                                     'Make_Id': int(input_.loc[a, 'Make_Id']),
                                     'Make': input_.loc[a, 'Make']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Year'] + ' - ' + input_.loc[a, 'Make'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Year_Id', 'Make_Id', 'Model_Id'], ascending=[False, True, True]).reset_index(drop=True)
output_correct.to_excel('./model-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Year_Id', 'Make_Id'], ascending=[False, True]).reset_index(drop=True)
    output_error.to_excel('./model_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1238

[ok] - 2023 - Volvo
[尝试次数：1] - [剩余数量：1228] - [当前时间：18:06:14]

[ok] - 2024 - Nissan
[尝试次数：1] - [剩余数量：1227] - [当前时间：18:06:14]

[ok] - 2023 - Ford
[尝试次数：1] - [剩余数量：1226] - [当前时间：18:06:14]

[ok] - 2023 - Jaguar
[尝试次数：1] - [剩余数量：1225] - [当前时间：18:06:14]

[ok] - 2023 - Lincoln
[尝试次数：1] - [剩余数量：1224] - [当前时间：18:06:14]

[ok] - 2023 - Jeep
[尝试次数：1] - [剩余数量：1223] - [当前时间：18:06:14]

[ok] - 2023 - Hyundai
[尝试次数：1] - [剩余数量：1222] - [当前时间：18:06:14]

[ok] - 2023 - Chrysler
[尝试次数：1] - [剩余数量：1221] - [当前时间：18:06:14]

[ok] - 2023 - Dodge
[尝试次数：1] - [剩余数量：1220] - [当前时间：18:06:14]

[ok] - 2023 - Kia
[尝试次数：1] - [剩余数量：1219] - [当前时间：18:06:14]

[ok] - 2023 - Honda
[尝试次数：1] - [剩余数量：1218] - [当前时间：18:06:15]

[ok] - 2023 - INFINITI
[尝试次数：1] - [剩余数量：1217] - [当前时间：18:06:15]

[ok] - 2023 - Mercedes-Benz
[尝试次数：1] - [剩余数量：1216] - [当前时间：18:06:15]

[ok] - 2023 - Volkswagen
[尝试次数：1] - [剩余数量：1215] - [当前时间：18:06:15]

[ok] - 2023 - Toyota
[尝试次数：1] - [剩余数量：1214] - [当前时间：18:06:16]

[ok] - 2023 - Mazda
[尝试次数：1] - [剩余数量：1

[ok] - 2018 - Porsche
[尝试次数：1] - [剩余数量：1096] - [当前时间：18:06:37]

[ok] - 2018 - Hyundai
[尝试次数：1] - [剩余数量：1095] - [当前时间：18:06:37]

[ok] - 2018 - Jaguar
[尝试次数：1] - [剩余数量：1094] - [当前时间：18:06:37]

[ok] - 2018 - Fiat
[尝试次数：1] - [剩余数量：1093] - [当前时间：18:06:37]

[ok] - 2018 - Kia
[尝试次数：1] - [剩余数量：1092] - [当前时间：18:06:38]

[ok] - 2018 - Subaru
[尝试次数：1] - [剩余数量：1091] - [当前时间：18:06:38]

[ok] - 2018 - Mini
[尝试次数：1] - [剩余数量：1090] - [当前时间：18:06:38]

[ok] - 2018 - BMW
[尝试次数：1] - [剩余数量：1089] - [当前时间：18:06:38]

[ok] - 2018 - Chrysler
[尝试次数：1] - [剩余数量：1088] - [当前时间：18:06:38]

[ok] - 2019 - Mitsubishi
[尝试次数：1] - [剩余数量：1087] - [当前时间：18:06:38]

[ok] - 2018 - Dodge
[尝试次数：1] - [剩余数量：1086] - [当前时间：18:06:39]

[ok] - 2018 - Volvo
[尝试次数：1] - [剩余数量：1085] - [当前时间：18:06:39]

[ok] - 2018 - Jeep
[尝试次数：1] - [剩余数量：1084] - [当前时间：18:06:39]

[ok] - 2018 - Buick
[尝试次数：1] - [剩余数量：1083] - [当前时间：18:06:39]

[ok] - 2018 - Chevrolet
[尝试次数：1] - [剩余数量：1082] - [当前时间：18:06:39]

[ok] - 2018 - Cadillac
[尝试次数：1] - [剩余数量：1081] - [当前时间：18:06

[ok] - 2014 - Porsche
[尝试次数：1] - [剩余数量：965] - [当前时间：18:07:04]

[ok] - 2015 - Ram
[尝试次数：1] - [剩余数量：964] - [当前时间：18:07:04]

[ok] - 2014 - Hyundai
[尝试次数：1] - [剩余数量：963] - [当前时间：18:07:04]

[ok] - 2014 - Fiat
[尝试次数：1] - [剩余数量：962] - [当前时间：18:07:04]

[ok] - 2015 - Toyota
[尝试次数：1] - [剩余数量：961] - [当前时间：18:07:05]

[ok] - 2014 - Land Rover
[尝试次数：1] - [剩余数量：960] - [当前时间：18:07:05]

[ok] - 2014 - Subaru
[尝试次数：1] - [剩余数量：959] - [当前时间：18:07:05]

[ok] - 2014 - Kia
[尝试次数：1] - [剩余数量：958] - [当前时间：18:07:06]

[ok] - 2014 - Jaguar
[尝试次数：1] - [剩余数量：957] - [当前时间：18:07:06]

[ok] - 2014 - Volvo
[尝试次数：1] - [剩余数量：956] - [当前时间：18:07:06]

[ok] - 2014 - Mini
[尝试次数：1] - [剩余数量：955] - [当前时间：18:07:06]

[ok] - 2014 - BMW
[尝试次数：1] - [剩余数量：954] - [当前时间：18:07:06]

[ok] - 2014 - Dodge
[尝试次数：1] - [剩余数量：953] - [当前时间：18:07:06]

[ok] - 2014 - Chrysler
[尝试次数：1] - [剩余数量：952] - [当前时间：18:07:06]

[ok] - 2014 - Buick
[尝试次数：1] - [剩余数量：951] - [当前时间：18:07:06]

[ok] - 2014 - Cadillac
[尝试次数：1] - [剩余数量：950] - [当前时间：18:07:07]

[ok] - 2014 - 

[ok] - 2011 - INFINITI
[尝试次数：1] - [剩余数量：832] - [当前时间：18:07:26]

[ok] - 2011 - Rolls-Royce
[尝试次数：1] - [剩余数量：831] - [当前时间：18:07:26]

[ok] - 2011 - Mitsubishi
[尝试次数：1] - [剩余数量：830] - [当前时间：18:07:26]

[ok] - 2011 - Audi
[尝试次数：1] - [剩余数量：829] - [当前时间：18:07:26]

[ok] - 2011 - Volkswagen
[尝试次数：1] - [剩余数量：828] - [当前时间：18:07:26]

[ok] - 2011 - Toyota
[尝试次数：1] - [剩余数量：827] - [当前时间：18:07:26]

[ok] - 2011 - Lexus
[尝试次数：1] - [剩余数量：826] - [当前时间：18:07:26]

[ok] - 2011 - Mazda
[尝试次数：1] - [剩余数量：825] - [当前时间：18:07:27]

[ok] - 2011 - Scion
[尝试次数：1] - [剩余数量：824] - [当前时间：18:07:27]

[ok] - 2011 - Smart
[尝试次数：1] - [剩余数量：823] - [当前时间：18:07:27]

[ok] - 2011 - Ram
[尝试次数：1] - [剩余数量：822] - [当前时间：18:07:27]

[ok] - 2010 - Suzuki
[尝试次数：1] - [剩余数量：821] - [当前时间：18:07:27]

[ok] - 2010 - Land Rover
[尝试次数：1] - [剩余数量：820] - [当前时间：18:07:27]

[ok] - 2010 - Hyundai
[尝试次数：1] - [剩余数量：819] - [当前时间：18:07:27]

[ok] - 2010 - Jaguar
[尝试次数：1] - [剩余数量：818] - [当前时间：18:07:28]

[ok] - 2010 - Kia
[尝试次数：1] - [剩余数量：817] - [当前时间：18:07:28]



[ok] - 2007 - Jeep
[尝试次数：1] - [剩余数量：700] - [当前时间：18:07:48]

[ok] - 2007 - Buick
[尝试次数：1] - [剩余数量：699] - [当前时间：18:07:48]

[ok] - 2007 - Cadillac
[尝试次数：1] - [剩余数量：698] - [当前时间：18:07:48]

[ok] - 2007 - Chrysler
[尝试次数：1] - [剩余数量：697] - [当前时间：18:07:48]

[ok] - 2007 - Chevrolet
[尝试次数：1] - [剩余数量：696] - [当前时间：18:07:48]

[ok] - 2007 - GMC
[尝试次数：1] - [剩余数量：695] - [当前时间：18:07:48]

[ok] - 2007 - Saturn
[尝试次数：1] - [剩余数量：694] - [当前时间：18:07:49]

[ok] - 2007 - Ford
[尝试次数：1] - [剩余数量：693] - [当前时间：18:07:49]

[ok] - 2007 - Mercury
[尝试次数：1] - [剩余数量：692] - [当前时间：18:07:49]

[ok] - 2007 - Acura
[尝试次数：1] - [剩余数量：691] - [当前时间：18:07:50]

[ok] - 2007 - Honda
[尝试次数：1] - [剩余数量：690] - [当前时间：18:07:50]

[ok] - 2007 - Mercedes-Benz
[尝试次数：1] - [剩余数量：689] - [当前时间：18:07:50]

[ok] - 2007 - Saab
[尝试次数：1] - [剩余数量：688] - [当前时间：18:07:50]

[ok] - 2007 - Nissan
[尝试次数：1] - [剩余数量：687] - [当前时间：18:07:50]

[ok] - 2007 - INFINITI
[尝试次数：1] - [剩余数量：686] - [当前时间：18:07:51]

[ok] - 2007 - Volkswagen
[尝试次数：1] - [剩余数量：685] - [当前时间：18:07:51]


[ok] - 2004 - Volkswagen
[尝试次数：1] - [剩余数量：568] - [当前时间：18:08:10]

[ok] - 2004 - Lexus
[尝试次数：1] - [剩余数量：567] - [当前时间：18:08:10]

[ok] - 2004 - Toyota
[尝试次数：1] - [剩余数量：566] - [当前时间：18:08:10]

[ok] - 2004 - Mazda
[尝试次数：1] - [剩余数量：565] - [当前时间：18:08:10]

[ok] - 2004 - Hummer
[尝试次数：1] - [剩余数量：564] - [当前时间：18:08:11]

[ok] - 2004 - Scion
[尝试次数：1] - [剩余数量：563] - [当前时间：18:08:11]

[ok] - 2003 - Hyundai
[尝试次数：1] - [剩余数量：562] - [当前时间：18:08:11]

[ok] - 2003 - Suzuki
[尝试次数：1] - [剩余数量：561] - [当前时间：18:08:11]

[ok] - 2003 - Avanti
[尝试次数：1] - [剩余数量：560] - [当前时间：18:08:11]

[ok] - 2003 - Land Rover
[尝试次数：1] - [剩余数量：559] - [当前时间：18:08:11]

[ok] - 2003 - Subaru
[尝试次数：1] - [剩余数量：558] - [当前时间：18:08:11]

[ok] - 2003 - Jaguar
[尝试次数：1] - [剩余数量：557] - [当前时间：18:08:12]

[ok] - 2003 - Kia
[尝试次数：1] - [剩余数量：556] - [当前时间：18:08:12]

[ok] - 2003 - Volvo
[尝试次数：1] - [剩余数量：555] - [当前时间：18:08:12]

[ok] - 2003 - Mini
[尝试次数：1] - [剩余数量：554] - [当前时间：18:08:12]

[ok] - 2003 - BMW
[尝试次数：1] - [剩余数量：553] - [当前时间：18:08:12]

[ok] - 2003

[ok] - 2000 - Acura
[尝试次数：1] - [剩余数量：433] - [当前时间：18:08:31]

[ok] - 2000 - Honda
[尝试次数：1] - [剩余数量：432] - [当前时间：18:08:31]

[ok] - 2000 - Saab
[尝试次数：1] - [剩余数量：431] - [当前时间：18:08:31]

[ok] - 2000 - Nissan
[尝试次数：1] - [剩余数量：430] - [当前时间：18:08:31]

[ok] - 2000 - INFINITI
[尝试次数：1] - [剩余数量：429] - [当前时间：18:08:32]

[ok] - 2000 - Audi
[尝试次数：1] - [剩余数量：428] - [当前时间：18:08:32]

[ok] - 2000 - Mitsubishi
[尝试次数：1] - [剩余数量：427] - [当前时间：18:08:32]

[ok] - 2000 - Lexus
[尝试次数：1] - [剩余数量：426] - [当前时间：18:08:32]

[ok] - 2000 - Volkswagen
[尝试次数：1] - [剩余数量：425] - [当前时间：18:08:32]

[ok] - 2000 - Toyota
[尝试次数：1] - [剩余数量：424] - [当前时间：18:08:32]

[ok] - 2000 - Mazda
[尝试次数：1] - [剩余数量：423] - [当前时间：18:08:32]

[ok] - 1999 - Suzuki
[尝试次数：1] - [剩余数量：422] - [当前时间：18:08:32]

[ok] - 1999 - Hyundai
[尝试次数：1] - [剩余数量：421] - [当前时间：18:08:33]

[ok] - 1999 - Land Rover
[尝试次数：1] - [剩余数量：420] - [当前时间：18:08:33]

[ok] - 1999 - Kia
[尝试次数：1] - [剩余数量：419] - [当前时间：18:08:33]

[ok] - 1999 - Volvo
[尝试次数：1] - [剩余数量：418] - [当前时间：18:08:33]

[ok] 

[ok] - 1995 - Suzuki
[尝试次数：1] - [剩余数量：299] - [当前时间：18:08:52]

[ok] - 1995 - Hyundai
[尝试次数：1] - [剩余数量：298] - [当前时间：18:08:52]

[ok] - 1995 - Kia
[尝试次数：1] - [剩余数量：297] - [当前时间：18:08:53]

[ok] - 1995 - BMW
[尝试次数：1] - [剩余数量：296] - [当前时间：18:08:53]

[ok] - 1995 - Volvo
[尝试次数：1] - [剩余数量：295] - [当前时间：18:08:53]

[ok] - 1995 - Chrysler
[尝试次数：1] - [剩余数量：294] - [当前时间：18:08:53]

[ok] - 1995 - Dodge
[尝试次数：1] - [剩余数量：293] - [当前时间：18:08:53]

[ok] - 1995 - Plymouth
[尝试次数：1] - [剩余数量：292] - [当前时间：18:08:53]

[ok] - 1995 - Eagle
[尝试次数：1] - [剩余数量：291] - [当前时间：18:08:53]

[ok] - 1995 - Jeep
[尝试次数：1] - [剩余数量：290] - [当前时间：18:08:53]

[ok] - 1995 - Buick
[尝试次数：1] - [剩余数量：289] - [当前时间：18:08:54]

[ok] - 1995 - Chevrolet
[尝试次数：1] - [剩余数量：288] - [当前时间：18:08:54]

[ok] - 1995 - Cadillac
[尝试次数：1] - [剩余数量：287] - [当前时间：18:08:54]

[ok] - 1996 - INFINITI
[尝试次数：1] - [剩余数量：286] - [当前时间：18:08:55]

[ok] - 1995 - GMC
[尝试次数：1] - [剩余数量：285] - [当前时间：18:08:55]

[ok] - 1995 - Geo
[尝试次数：1] - [剩余数量：284] - [当前时间：18:08:55]

[ok] - 1995 - 

[ok] - 1991 - Toyota
[尝试次数：1] - [剩余数量：165] - [当前时间：18:09:13]

[ok] - 1990 - Suzuki
[尝试次数：1] - [剩余数量：164] - [当前时间：18:09:14]

[ok] - 1991 - Mazda
[尝试次数：1] - [剩余数量：163] - [当前时间：18:09:14]

[ok] - 1990 - Volvo
[尝试次数：1] - [剩余数量：162] - [当前时间：18:09:14]

[ok] - 1990 - Dodge
[尝试次数：1] - [剩余数量：161] - [当前时间：18:09:14]

[ok] - 1990 - BMW
[尝试次数：1] - [剩余数量：160] - [当前时间：18:09:14]

[ok] - 1990 - Chrysler
[尝试次数：1] - [剩余数量：159] - [当前时间：18:09:14]

[ok] - 1990 - Eagle
[尝试次数：1] - [剩余数量：158] - [当前时间：18:09:15]

[ok] - 1990 - Jeep
[尝试次数：1] - [剩余数量：157] - [当前时间：18:09:15]

[ok] - 1990 - Plymouth
[尝试次数：1] - [剩余数量：156] - [当前时间：18:09:15]

[ok] - 1990 - Buick
[尝试次数：1] - [剩余数量：155] - [当前时间：18:09:15]

[ok] - 1990 - Cadillac
[尝试次数：1] - [剩余数量：154] - [当前时间：18:09:15]

[ok] - 1990 - Chevrolet
[尝试次数：1] - [剩余数量：153] - [当前时间：18:09:15]

[ok] - 1990 - GMC
[尝试次数：1] - [剩余数量：152] - [当前时间：18:09:15]

[ok] - 1990 - Oldsmobile
[尝试次数：1] - [剩余数量：151] - [当前时间：18:09:16]

[ok] - 1990 - Ford
[尝试次数：1] - [剩余数量：150] - [当前时间：18:09:16]

[ok] - 199

[ok] - 1983 - Buick
[尝试次数：1] - [剩余数量：29] - [当前时间：18:09:40]

[ok] - 1983 - Cadillac
[尝试次数：1] - [剩余数量：28] - [当前时间：18:09:40]

[ok] - 1983 - Chevrolet
[尝试次数：1] - [剩余数量：27] - [当前时间：18:09:40]

[ok] - 1983 - Pontiac
[尝试次数：1] - [剩余数量：26] - [当前时间：18:09:41]

[ok] - 1983 - GMC
[尝试次数：1] - [剩余数量：25] - [当前时间：18:09:41]

[ok] - 1983 - Oldsmobile
[尝试次数：1] - [剩余数量：24] - [当前时间：18:09:41]

[ok] - 1983 - Ford
[尝试次数：1] - [剩余数量：23] - [当前时间：18:09:41]

[ok] - 1983 - Toyota
[尝试次数：1] - [剩余数量：22] - [当前时间：18:09:42]

[ok] - 1983 - Mercury
[尝试次数：1] - [剩余数量：21] - [当前时间：18:09:42]

[ok] - 1982 - Chrysler
[尝试次数：1] - [剩余数量：20] - [当前时间：18:09:42]

[ok] - 1982 - BMW
[尝试次数：1] - [剩余数量：19] - [当前时间：18:09:42]

[ok] - 1982 - Plymouth
[尝试次数：1] - [剩余数量：18] - [当前时间：18:09:42]

[ok] - 1982 - Dodge
[尝试次数：1] - [剩余数量：17] - [当前时间：18:09:42]

[ok] - 1982 - Cadillac
[尝试次数：1] - [剩余数量：16] - [当前时间：18:09:42]

[ok] - 1982 - Buick
[尝试次数：1] - [剩余数量：15] - [当前时间：18:09:42]

[ok] - 1982 - Chevrolet
[尝试次数：1] - [剩余数量：14] - [当前时间：18:09:42]

[ok] - 1982 - O